In [17]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from typing import Optional, List,Tuple,Dict

# Phone Ranking Problem

Choosing a new phone is often quite a daunting task, with many different models from different companies and varying prices.

In [2]:
data = pd.read_csv("./data/dataset.csv")
prefs = pd.read_csv("./data/preferences.csv")

In [3]:
data

,name,price-cost,ram-gain,screen-gain,disk-gain,oled-gain,os-gain,foldable-gain
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0


In [4]:
crit_types = [True if crit.split("-")[1]=="gain" else False for crit in data.drop(columns="name").columns ]
crit_types

[False, True, True, True, True, True, True]

In [11]:
data.columns = list(map(lambda s: s.split("-")[0],data.columns))
data

,name,price,ram,screen,disk,oled,os,foldable
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0



1. What is the domain of the problem about?
	
	The problem involves constinst of various phones and some of their parameters. 
	Price is given in złoty's, Ram in GB, Screen in inches, disk in GB, Oled and Foldable are binary variables and its beneficial for the phone to have them.
	OS ranks the operating systems with 0 being iOS, 1 being other, 2 being HarmonyOS and 3 being Android.

2. What is the source of the data?

	The data is taken from the internet from various online shops that offer these phones.

In [ ]:
def type_1(d):
	return 1 if d > 0 else 0

def type_5(d,q:float,p:float):
	if d > p:
		return 1
	elif d < q:
		return 0
	else: 
		return (d-q)/(p-q)

In [ ]:
def rank_string(ranking,names):
	_str = ""
	for i,n in sorted(zip(ranking,names),key=lambda a:a[0]):
		_str += f"{i}:\t{n}"
	return _str

def show_rank(ranking,names):
	print(rank_string(ranking,names))

In [ ]:
class Promethee:
	def __init__(self,data,criterion_type,criterion_weights=None,discrimination_thresholds=None):
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.ranking = [i for i in range(len(self.names))]
		self.is_crit_gain = criterion_type
		self.criterion_weighs = criterion_weights
		self.criterion_funcs = self._init_crit_funcs(discrimination_thresholds)

	def _init_crit_funcs(self,disc_thresh):
		return [lambda a,b: type5(a-b,q,p) if self.is_crit_gain else type5(b-a,q,p) for q,p in disc_thresh]
		

	def _cp_index(self,data:pd.DataFrame,criterion_weights:list[float]=None,criterion_func:list[callable]=None)->np.array:

		n = len(data)
		matrix = np.zeros((n,n))

		if criterion_weights==None:
			criterion_weights = [1 for _ in data.columns]

		if criterion_func == None:
			criterion_func = [type_1 for _ in criterion_weights]

		for c,criterion in enumerate(data.columns):
			for i,a in enumerate(data[criterion]):
				for j,b in enumerate(data[criterion][i+1:],i+1):
					pi = criterion_func[c]
					w = criterion_weights[c]
					matrix[i][j] += w * pi(a,b)
					matrix[j][i] += w * pi(b,a)
		
		return matrix/sum(criterion_weights)
	
	def _flow(cpi:np.array):
		positive_flow = np.sum(np.triu(cpi),axis=1)
		negative_flow = np.sum(np.triu(cpi.T),axis=1)

		return positive_flow,negative_flow
	
	def __str__(self):
		return rank_string(self.ranking,self.names)

In [ ]:
class Promethee1(Promethee):
	def rank(self):
		cpi = self._cp_index(data)
		pos_flow,neg_flow = self._flow(cpi)

		self.pos_rank = np.argsort(-pos_flow)
		self.neg_rank = np.argsort(neg_flow)

		

		return self.ranking

In [ ]:
class Promethee2(Promethee):
	def rank(self):
		cpi = self._cp_index(data)
		pos_flow,neg_flow = self._flow(cpi)

		f = pos_flow-neg_flow

		self.ranking = np.argsort(-f)

		return self.ranking

In [ ]:
estimated_weights = [7,5,3,4,4,8,5]

In [ ]:
cpi = cp_index(data.drop(columns="name"))
plt.imshow(cpi, interpolation='nearest')
plt.show()

In [ ]:
flow(cpi)

In [ ]:
show_rank(prom2_rank,data["name"])

In [ ]:
show_rank(prom1_pos,data["name"])

In [ ]:
show_rank(prom1_neg,data["name"])

In [83]:
class ElectreTriB:
	def __init__(self,data:pd.DataFrame,criterion_type:list[bool],criterion_weights:list[float],class_profiles:np.array, pref_indif_veto:np.array):
		"""
			class_profiles should be (criterion x profiles)

			pref_indif_veto should be(criterion x profiles x 3)
		"""
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.is_crit_gain = criterion_type
		self.class_profiles = class_profiles
		self.pref_indif_veto = pref_indif_veto
		self.weights = criterion_weights

		self.b = len(self.class_profiles)
		self.g = len(self.data.columns)
		self.a = len(self.data)

	def _marginals(self,thresholds):
		matrix = np.zeros((self.g,self.b,self.a))

		for g_i,g in enumerate(self.data.columns):
			for a_i in range(self.a):
				for b_j in range(self.b):
					d = self.data[g][a_i] - self.class_profiles[g_i][b_j]
					if self.is_crit_gain[g_i]:
						d *= -1
					q,p = thresholds[g_i][b_j],self.pref_indif_veto[g_i][b_j][1]
					matrix[g_i][b_j][a_i] = type_5(d,q,p)

		return matrix

	def _marginal_concordance(self):
		self.mconc = self._marginals(self.pref_indif_veto[:,:,0])

	def _comprehensive_concordance(self):
		self.cconc = np.sum(np.array(self.criterion_weights) * self.mconc,axis=0)

	def _marginal_discordance(self):
		self.mdisc = self._marginals(self.pref_indif_veto[:,:,2])

	def _outranking_credibility(self):
		self.out_cred = np.zeros(self.a,self.b)

		for a_i in range(self.a):
			for b_j in range(self.b):
				_C = self.cconc[b_j][a_i]
				f = [f_j for f_j in self.mdisc[:,b_j,a_i] if f_j > self.cconc[b_j][a_i]]
				prod = np.prod([(1-f_j)/(1-_C) for f_j in f])
				self.out_cred[a_i][b_j] = _C * prod

	def run(self):
		self._marginal_concordance()
		self._marginal_discordance()
		self._comprehensive_concordance()
		self._outranking_credibility()

: 

In [78]:
def srf(rank:List[Optional[Tuple[str]]])->Dict[str,float]:
    """
        blank cards are none
    """

    n = len(rank)
    z = n-1
    blanks_below = rank.count(None)
    t = n-blanks_below
    r = []

    for i,elem in enumerate(rank):
        if elem is None:
            continue
        r.append(n-i)
    
    weights = np.array([1+((z-1)*((r_t-1)/(r[0]-1))) for r_t in r])

    criterions = filter(lambda a: not a is None,rank)
    mapping = {}
    for i,crits in enumerate(criterions):
        for crit in crits:
            mapping[crit] = weights[i]
    
    _sum = sum(mapping.values())
    mapping = dict([(k,v/_sum) for k,v in mapping.items()])

    return mapping

In [79]:
ranking = [
    ("price",),
    None,
    None,
    ("os",),
    None,
    None,
    ("foldable",),
    ("oled","screen"),
    ("ram","disk")]

In [80]:
crit_weights = srf(ranking)

{'price': 0.3657142857142857,
 'os': 0.24571428571428572,
 'foldable': 0.12571428571428572,
 'oled': 0.08571428571428572,
 'screen': 0.08571428571428572,
 'ram': 0.045714285714285714,
 'disk': 0.045714285714285714}

In [ ]:
ElectreTriB(data,crit_types,crit_weights)